In [1]:
import pandas as pd

In [2]:
import mysql.connector as mc
from mysql.connector import Error
import mysql.connector

## Conexión SQL

In [3]:
# Conexión a SQL usando pymysql
def serverCon(host_name, user_name, password, db, pt):
    import pymysql
    connection = None
    try:
        connection = pymysql.connect(
            host=host_name,
            user=user_name,
            password=password,
            database=db,
            port=int(pt),
            charset='utf8mb4',
            cursorclass=pymysql.cursors.DictCursor,
            autocommit=True
        )
        print('Conexión exitosa')
    except Exception as err:
        print(f"Error al conectar: '{err}'")
    return connection

In [4]:
#Ejecución Query
def executeQuery(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print('Query Ejecutado')
    except Error as err:
        print(f"Error: '{err}'")

In [5]:
conect = serverCon('192.168.200.212', 'industrial', 'p@ss4DB', 'jornada_ii', '3306') #Puerto 3306

Conexión exitosa


In [6]:
# Listar todas las tablas de la base de datos
def get_table_names(connection):
    with connection.cursor() as cursor:
        cursor.execute("SHOW TABLES;")
        tables = [row[list(row.keys())[0]] for row in cursor.fetchall()]
    return tables

tablas = get_table_names(conect)
tablas

['actividades',
 'asistencias',
 'configuracion_sistema',
 'equipos_concurso',
 'estados_mexico',
 'inscripciones_workshop',
 'participantes',
 'v_asistencias_stats',
 'v_asistencias_workshops',
 'v_participantes_completos',
 'v_workshop_stats']

## Dashboard básico para participantes

In [11]:
import os
from pathlib import Path
import pymysql
import pandas as pd

# Conexión pymysql directa (ajusta los valores si es necesario)
conn = pymysql.connect(
    host='192.168.200.212',
    user='industrial',
    password='p@ss4DB',
    database='jornada_ii',
    port=3306,
    charset='utf8mb4',
    autocommit=True,
    cursorclass=pymysql.cursors.DictCursor
)

output_dir = Path('MultiPage App/datos')
output_dir.mkdir(parents=True, exist_ok=True)

# --- Consulta y guardado de participantes ---
query_participantes = '''
    SELECT 
        id,
        CONCAT(primer_nombre, ' ', COALESCE(segundo_nombre, ''), ' ', apellido_paterno, ' ', apellido_materno) AS nombre_completo,
        email,
        categoria,
        programa,
        brazalete
    FROM participantes
    ORDER BY creado DESC
'''
with conn.cursor() as cur:
    cur.execute(query_participantes)
    rows = cur.fetchall()
df_participantes = pd.DataFrame(rows)
print('Participantes:', df_participantes.shape)
display(df_participantes.head())
df_participantes.to_csv(output_dir / 'participantes.csv', index=False)

# --- Consulta y guardado de inscripciones_workshop ---
query_insc = '''
    SELECT iw.id, p.email AS participante_email, a.codigo AS actividad_codigo, iw.estado, iw.creado
    FROM inscripciones_workshop iw
    JOIN participantes p ON iw.participante_id = p.id
    JOIN actividades a ON iw.actividad_id = a.id
    ORDER BY iw.creado DESC
'''
with conn.cursor() as cur:
    cur.execute(query_insc)
    rows = cur.fetchall()
df_insc = pd.DataFrame(rows)
print('Inscripciones:', df_insc.shape)
display(df_insc.head())
df_insc.to_csv(output_dir / 'inscripciones_workshop.csv', index=False)

# --- Consulta y guardado de equipos_concurso ---
query_equipos = '''
    SELECT id, nombre_equipo, estado_id, email_capitan, estado_registro, activo, fecha_registro, fecha_confirmacion
    FROM equipos_concurso
    ORDER BY fecha_registro DESC
'''
with conn.cursor() as cur:
    cur.execute(query_equipos)
    rows = cur.fetchall()
df_equipos = pd.DataFrame(rows)
print('Equipos:', df_equipos.shape)
display(df_equipos.head())
df_equipos.to_csv(output_dir / 'equipos_concurso.csv', index=False)

# --- Consulta y guardado de asistencias ---
query_asist = '''
    SELECT a.id, p.email AS participante_email, act.codigo AS actividad_codigo, a.estado, a.modo_asistencia, a.fecha_asistencia, a.notas
    FROM asistencias a
    JOIN participantes p ON a.participante_id = p.id
    JOIN actividades act ON a.actividad_id = act.id
    ORDER BY a.fecha_asistencia DESC
'''
with conn.cursor() as cur:
    cur.execute(query_asist)
    rows = cur.fetchall()
df_asist = pd.DataFrame(rows)
print('Asistencias:', df_asist.shape)
display(df_asist.head())
df_asist.to_csv(output_dir / 'asistencias.csv', index=False)

conn.close()

Participantes: (245, 6)


,id,nombre_completo,email,categoria,programa,brazalete
0,246,Brayan Daniel Torres Calderon,250300906@ucaribe.edu.mx,Estudiante,Ingeniería Industrial,110.0
1,244,Adolfo Martínez Romero,230300962@ucaribe.edu.mx,Estudiante,Ingeniería Industrial,NaN
2,243,Vanessa Muñoz Romero,2303000931@ucaribe.edu.mx,Estudiante,Ingeniería Industrial,NaN
3,242,Zuñigan Leon Héctor Julián,230300956@ucaribe.edu.mx,Estudiante,Ingeniería Industrial,NaN
4,241,Mateo Hernández Mendoza,250300928@ucaribe.edu.mx,Estudiante,Ingeniería Industrial,NaN


Inscripciones: (227, 5)


,id,participante_email,actividad_codigo,estado,creado
0,2235,240300997@ucaribe.edu.mx,W5,cancelado,2025-09-22 18:25:31
1,2234,250300925@ucaribe.edu.mx,W1,inscrito,2025-09-22 18:16:16
2,2233,250300899@ucaribe.edu.mx,W1,inscrito,2025-09-22 18:01:26
3,2232,250300928@ucaribe.edu.mx,W1,inscrito,2025-09-22 18:00:07
4,2231,240300927@ucaribe.edu.mx,W1,inscrito,2025-09-22 17:39:10


Equipos: (2, 8)


,id,nombre_equipo,estado_id,email_capitan,estado_registro,activo,fecha_registro,fecha_confirmacion
0,2,Equipo Veracruz,30,240300886@ucaribe.edu.mx,pendiente,1,2025-09-22 13:59:16,None
1,1,Equipo Yucatán,31,250300917@ucaribe.edu.mx,pendiente,1,2025-09-22 12:58:52,None


Asistencias: (0, 0)


""
